In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
def assign(data, column, ran, values, show=True, con=False, cat=True):
    if bool(con):
        conditions=con
    else:
        conditions = []
        ran = list(set(ran))
        ran.sort(reverse=True)
        for i in range(len(ran) - 1):
            conditions.append(np.where(np.logical_and(data[column].value_counts() <= ran[i],
                                                      data[column].value_counts() > ran[i+1]),
                                       data[column].value_counts().index,
                                       np.nan))
    if show:
        print(f'Number of condition(s): {len(conditions)}')
    
    for i, j in zip(conditions, values):
        index = data[data[column].isin(i)].index
        data.loc[index, column] = j
        if show:
            print(f'{len(index)} data in column {column} have been replaced by value {j}')
    if show:
        print(data[column].value_counts())
    if cat:
        data[column] = data[column].astype('category')
    return

In [3]:
df = pd.read_excel('cctx_train.xlsx')
df.dropna(inplace=True)
df.drop(['Transaction_ID', 'Transaction_Flag', 'Bank_ID'], axis=1, inplace=True)
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 10484 entries, 0 to 10499
Data columns (total 18 columns):
 #   Column                         Non-Null Count  Dtype         
---  ------                         --------------  -----         
 0   Transaction_Date               10484 non-null  datetime64[ns]
 1   Transaction_Type               10484 non-null  object        
 2   Transaction_Amount             10484 non-null  float64       
 3   CC_ID                          10484 non-null  object        
 4   Card_Type                      10484 non-null  object        
 5   Card_Holder                    10484 non-null  int64         
 6   Channel_ID                     10484 non-null  int64         
 7   Merchant_ID                    10484 non-null  object        
 8   Country_ID                     10484 non-null  object        
 9   City_ID                        10484 non-null  object        
 10  EDC_Type                       10484 non-null  object        
 11  EDC_Location   

In [4]:
df.head()

,Transaction_Date,Transaction_Type,Transaction_Amount,CC_ID,Card_Type,Card_Holder,Channel_ID,Merchant_ID,Country_ID,City_ID,EDC_Type,EDC_Location,EDC_Owner,Average_Transaction_Amount,Maximum_Transaction_Amount,Minimum_Transaction_Amount,Average_Transaction_Frequency,Fraud_Status
0,2018-01-01 01:48:50.951,T08,50000.0,CCID5563,CC11,2,1,M0001,CTY06,CTY06-133,EDC0885,LEDC3703,OEDC0377,661855.03,11000000.0,24212.0,2.91,0
1,2018-01-01 09:08:52.666,T01,1000000.0,CCID4598,CC08,2,1,M0001,CTY06,CTY06-129,EDC0565,LEDC3205,OEDC0377,2171907.10,28555000.0,100000.0,3.12,1
2,2018-01-01 09:08:52.666,T15,1000000.0,CCID2839,CC08,2,1,M0001,CTY06,CTY06-073,EDC4639,LEDC3081,OEDC0377,1714437.98,8500000.0,50.0,3.58,0
3,2018-01-01 09:45:55.969,T15,1000000.0,CCID2968,CC08,2,1,M0001,CTY06,CTY06-129,EDC3918,LEDC1993,OEDC0377,10163708.23,100000000.0,63000.0,2.57,0
4,2018-01-01 23:41:59.228,T10,2500000.0,CCID0176,CC12,2,1,M0001,CTY06,CTY06-133,EDC2863,LEDC2062,OEDC0377,690066.65,3523000.0,26500.0,4.23,0


In [5]:
df['City_ID'].value_counts()

CTY06-023    4247
CTY06-133    1276
CTY06-181     800
CTY06-171     440
CTY06-186     337
             ... 
CTY06-106       1
CTY09-002       1
CTY06-161       1
CTY06-078       1
CTY06-019       1
Name: City_ID, Length: 227, dtype: int64

In [6]:
df['City_ID'].unique()

array(['CTY06-133', 'CTY06-129', 'CTY06-073', 'CTY06-023', 'CTY06-181',
       'CTY06-083', 'CTY06-173', 'CTY06-149', 'CTY06-004', 'CTY06-194',
       'CTY06-171', 'CTY06-096', 'CTY06-035', 'CTY06-186', 'CTY06-174',
       'CTY06-092', 'CTY06-029', 'CTY06-128', 'CTY06-101', 'CTY06-087',
       'CTY06-046', 'CTY06-197', 'CTY06-125', 'CTY06-056', 'CTY06-089',
       'CTY06-058', 'CTY06-099', 'CTY06-005', 'CTY06-180', 'CTY06-112',
       'CTY06-151', 'CTY06-176', 'CTY13-003', 'CTY06-049', 'CTY06-175',
       'CTY06-071', 'CTY06-140', 'CTY06-121', 'CTY06-090', 'CTY06-115',
       'CTY06-110', 'CTY06-195', 'CTY06-120', 'CTY06-164', 'CTY06-205',
       'CTY06-179', 'CTY06-126', 'CTY06-152', 'CTY06-039', 'CTY06-075',
       'CTY06-077', 'CTY06-166', 'CTY06-113', 'CTY06-148', 'CTY06-190',
       'CTY06-082', 'CTY06-067', 'CTY06-137', 'CTY06-147', 'CTY06-167',
       'CTY06-001', 'CTY06-097', 'CTY06-072', 'CTY06-086', 'CTY06-191',
       'CTY06-124', 'CTY06-189', 'CTY06-013', 'CTY06-185', 'CTY0

In [7]:
df['City_ID'].str.contains(pat='CTY06').value_counts()

True     10448
False       36
Name: City_ID, dtype: int64

In [8]:
df['Country_ID'].str.contains(pat='CTY06').value_counts()

True     10448
False       36
Name: Country_ID, dtype: int64

In [9]:
(df['City_ID'].str.contains(pat='CTY06') == 
 df['Country_ID'].str.contains(pat='CTY06')).value_counts()

True    10484
dtype: int64

In [10]:
col = 'Country_ID'
ran = [0, 100, 20_000]
val = ['Indonesia', 'Asing']
assign(df, col, ran, val)

Number of condition(s): 2
10448 data in column Country_ID have been replaced by value Indonesia
36 data in column Country_ID have been replaced by value Asing
Indonesia    10448
Asing           36
Name: Country_ID, dtype: int64


In [11]:
df['EDC_Type'].value_counts()

EDC0002    705
EDC1846     25
EDC0729     17
EDC1155     17
EDC0798     16
          ... 
EDC2558      1
EDC4631      1
EDC4342      1
EDC4846      1
EDC4477      1
Name: EDC_Type, Length: 4787, dtype: int64

In [12]:
col = 'EDC_Type'
ran = [0, 10, 100, 1_000]
val = ['Bank', 'Mobile', 'GPRS']
assign(df, col, ran, val)

Number of condition(s): 3
705 data in column EDC_Type have been replaced by value Bank
365 data in column EDC_Type have been replaced by value Mobile
9414 data in column EDC_Type have been replaced by value GPRS
GPRS      9414
Bank       705
Mobile     365
Name: EDC_Type, dtype: int64


In [13]:
df['EDC_Owner'].value_counts()

OEDC0377    8312
OEDC0378      15
OEDC0212       6
OEDC1363       6
OEDC0886       6
            ... 
OEDC0530       1
OEDC0721       1
OEDC1412       1
OEDC1497       1
OEDC1557       1
Name: EDC_Owner, Length: 1460, dtype: int64

In [14]:
col = 'EDC_Owner'
ran = [0, 100, 10_000]
val = ['Pemilik', 'Sewa']
assign(df, col, ran, val)

Number of condition(s): 2
8312 data in column EDC_Owner have been replaced by value Pemilik
2172 data in column EDC_Owner have been replaced by value Sewa
Pemilik    8312
Sewa       2172
Name: EDC_Owner, dtype: int64


In [15]:
df['Transaction_Type'].value_counts()

T01    2866
T15    2447
T02    1323
T08    1072
T13     768
T06     675
T14     509
T04     263
T03     206
T10     112
T12      77
T11      50
T20      29
T17      27
T05      22
T16      12
T09       8
T19       7
T07       7
T18       4
Name: Transaction_Type, dtype: int64

In [16]:
col = 'Transaction_Type'
ran = [0, 100, 1_000, 10_000]
val = ['Sering', 'Umum', 'Jarang']
assign(df, col, ran, val)

Number of condition(s): 3
7708 data in column Transaction_Type have been replaced by value Sering
2533 data in column Transaction_Type have been replaced by value Umum
243 data in column Transaction_Type have been replaced by value Jarang
Sering    7708
Umum      2533
Jarang     243
Name: Transaction_Type, dtype: int64


In [17]:
df['Card_Type'].value_counts()

CC11    3835
CC08    2106
CC09    1379
CC00    1260
CC10    1092
CC04     237
CC02     157
CC01     155
CC05     145
CC12      54
CC13      36
CC03      20
CC06       7
CC07       1
Name: Card_Type, dtype: int64

In [18]:
cc = df.groupby(['Card_Type']).agg({'Maximum_Transaction_Amount': ['count', 'std', 'min', 'max']}).sort_values(('Maximum_Transaction_Amount', 'max'), ascending=False)
cc

Maximum_Transaction_Amount                                       
                               count           std         min          max
Card_Type                                                                  
CC02                             157  2.455402e+07   1000000.0  100000000.0
CC05                             145  1.118061e+07    500000.0  100000000.0
CC08                            2106  2.571725e+07    700000.0  100000000.0
CC04                             237  1.302443e+07    500000.0   75000000.0
CC10                            1092  6.384819e+06    100000.0   69750000.0
CC11                            3835  9.627690e+06    243000.0   69750000.0
CC00                            1260  9.274332e+06    300000.0   58090000.0
CC09                            1379  5.554187e+06     38000.0   56161050.0
CC01                             155  8.910502e+06    500000.0   50000000.0
CC03                              20  1.601625e+07    500000.0   50000000.0
CC06                               7  1.881146e+07   1500000.0   50000000.0
CC13                              36  9.231345e+06    100000.0   50000000.0
CC12                              54  8.975019e+06    750000.0   46000000.0
CC07                               1           NaN  12500000.0   12500000.0

In [19]:
con = [cc[ :3].index,
       cc[3:8].index,
       cc[8: ].index]
con

[Index(['CC02', 'CC05', 'CC08'], dtype='object', name='Card_Type'),
 Index(['CC04', 'CC10', 'CC11', 'CC00', 'CC09'], dtype='object', name='Card_Type'),
 Index(['CC01', 'CC03', 'CC06', 'CC13', 'CC12', 'CC07'], dtype='object', name='Card_Type')]

In [20]:
col = 'Card_Type'
ran = False
val = ['Platinum', 'Gold', 'Silver']
assign(df, col, ran, val, con=con)

Number of condition(s): 3
2408 data in column Card_Type have been replaced by value Platinum
7803 data in column Card_Type have been replaced by value Gold
273 data in column Card_Type have been replaced by value Silver
Gold        7803
Platinum    2408
Silver       273
Name: Card_Type, dtype: int64


In [21]:
df['Channel_ID'].value_counts()

1    8312
2    1462
5     680
3      25
4       5
Name: Channel_ID, dtype: int64

In [22]:
col = 'Channel_ID'
ran = [0, 8_000, 10_000]
val = ['Primer', 'Sekunder']
assign(df, col, ran, val)

Number of condition(s): 2
8312 data in column Channel_ID have been replaced by value Primer
2172 data in column Channel_ID have been replaced by value Sekunder
Primer      8312
Sekunder    2172
Name: Channel_ID, dtype: int64


In [23]:
df['Merchant_ID'].value_counts()

M0001    9022
M0243       6
M0063       6
M0140       6
M0100       6
         ... 
M0873       1
M0697       1
M0848       1
M1071       1
M0597       1
Name: Merchant_ID, Length: 974, dtype: int64

In [24]:
col = 'Merchant_ID'
ran = [0, 9_000, 10_000]
val = ['Primer', 'Sekunder']
assign(df, col, ran, val)

Number of condition(s): 2
9022 data in column Merchant_ID have been replaced by value Primer
1462 data in column Merchant_ID have been replaced by value Sekunder
Primer      9022
Sekunder    1462
Name: Merchant_ID, dtype: int64


In [25]:
df['Card_Holder'].value_counts()

2    9774
1     710
Name: Card_Holder, dtype: int64

In [26]:
df.loc[df[df['Card_Holder'] == 2].index, 'Card_Holder'] = 'Primer'
df.loc[df[df['Card_Holder'] == 1].index, 'Card_Holder'] = 'Sekunder'
df['Card_Holder'] = df['Card_Holder'].astype('category')
df.info(memory_usage='deep')

<class 'pandas.core.frame.DataFrame'>
Int64Index: 10484 entries, 0 to 10499
Data columns (total 18 columns):
 #   Column                         Non-Null Count  Dtype         
---  ------                         --------------  -----         
 0   Transaction_Date               10484 non-null  datetime64[ns]
 1   Transaction_Type               10484 non-null  category      
 2   Transaction_Amount             10484 non-null  float64       
 3   CC_ID                          10484 non-null  object        
 4   Card_Type                      10484 non-null  category      
 5   Card_Holder                    10484 non-null  category      
 6   Channel_ID                     10484 non-null  category      
 7   Merchant_ID                    10484 non-null  category      
 8   Country_ID                     10484 non-null  category      
 9   City_ID                        10484 non-null  object        
 10  EDC_Type                       10484 non-null  category      
 11  EDC_Location   

In [27]:
df.head()

,Transaction_Date,Transaction_Type,Transaction_Amount,CC_ID,Card_Type,Card_Holder,Channel_ID,Merchant_ID,Country_ID,City_ID,EDC_Type,EDC_Location,EDC_Owner,Average_Transaction_Amount,Maximum_Transaction_Amount,Minimum_Transaction_Amount,Average_Transaction_Frequency,Fraud_Status
0,2018-01-01 01:48:50.951,Sering,50000.0,CCID5563,Gold,Primer,Primer,Primer,Indonesia,CTY06-133,GPRS,LEDC3703,Pemilik,661855.03,11000000.0,24212.0,2.91,0
1,2018-01-01 09:08:52.666,Sering,1000000.0,CCID4598,Platinum,Primer,Primer,Primer,Indonesia,CTY06-129,GPRS,LEDC3205,Pemilik,2171907.10,28555000.0,100000.0,3.12,1
2,2018-01-01 09:08:52.666,Sering,1000000.0,CCID2839,Platinum,Primer,Primer,Primer,Indonesia,CTY06-073,GPRS,LEDC3081,Pemilik,1714437.98,8500000.0,50.0,3.58,0
3,2018-01-01 09:45:55.969,Sering,1000000.0,CCID2968,Platinum,Primer,Primer,Primer,Indonesia,CTY06-129,GPRS,LEDC1993,Pemilik,10163708.23,100000000.0,63000.0,2.57,0
4,2018-01-01 23:41:59.228,Umum,2500000.0,CCID0176,Silver,Primer,Primer,Primer,Indonesia,CTY06-133,GPRS,LEDC2062,Pemilik,690066.65,3523000.0,26500.0,4.23,0


In [28]:
pd.get_dummies(df.drop(['Transaction_Date', 'CC_ID', 'City_ID', 'EDC_Location'], axis=1))

,Transaction_Amount,Average_Transaction_Amount,Maximum_Transaction_Amount,Minimum_Transaction_Amount,Average_Transaction_Frequency,Fraud_Status,Transaction_Type_Jarang,Transaction_Type_Sering,Transaction_Type_Umum,Card_Type_Gold,...,Channel_ID_Sekunder,Merchant_ID_Primer,Merchant_ID_Sekunder,Country_ID_Asing,Country_ID_Indonesia,EDC_Type_Bank,EDC_Type_GPRS,EDC_Type_Mobile,EDC_Owner_Pemilik,EDC_Owner_Sewa
0,50000.0,661855.03,11000000.0,24212.0,2.91,0,0,1,0,1,...,0,1,0,0,1,0,1,0,1,0
1,1000000.0,2171907.10,28555000.0,100000.0,3.12,1,0,1,0,0,...,0,1,0,0,1,0,1,0,1,0
2,1000000.0,1714437.98,8500000.0,50.0,3.58,0,0,1,0,0,...,0,1,0,0,1,0,1,0,1,0
3,1000000.0,10163708.23,100000000.0,63000.0,2.57,0,0,1,0,0,...,0,1,0,0,1,0,1,0,1,0
4,2500000.0,690066.65,3523000.0,26500.0,4.23,0,0,0,1,0,...,0,1,0,0,1,0,1,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10495,5000000.0,1667486.67,20000000.0,50000.0,4.15,0,0,0,1,0,...,0,1,0,0,1,0,1,0,1,0
10496,4800000.0,2530285.33,5000000.0,1000000.0,1.64,0,0,0,1,1,...,0,1,0,0,1,0,1,0,1,0
10497,2500000.0,1225000.00,10000000.0,100000.0,1.97,0,0,0,1,1,...,0,1,0,0,1,0,1,0,1,0
10498,1100000.0,820036.79,4000000.0,50000.0,3.16,0,0,0,1,1,...,0,1,0,0,1,0,1,0,1,0


<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=d45eaae7-31d4-4f2e-97ba-c3c70ea6b56e' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>